In [18]:
import psycopg2
import csv
import boto3
import configparser

In [19]:
data_parser = configparser.ConfigParser()

In [20]:
data_parser.read('pipeline.conf')

['pipeline.conf']

In [21]:
host = data_parser.get("postgres_config" , "host")

In [22]:
port = data_parser.get("postgres_config" , "port")
user_name = data_parser.get("postgres_config" , "username")
password = data_parser.get("postgres_config" , "password")
db = data_parser.get("postgres_config" , "database")

In [23]:
conn = psycopg2.connect("dbname=" + db
+ " user=" + user_name
+ " password=" + password
+ " host=" + host,
port = port)

In [24]:
query = """select * from orders"""

In [25]:
curser = conn.cursor()

In [26]:
curser.execute(query)

In [27]:
result = curser.fetchall()

In [28]:
with open("orders.csv" , "w") as f:
    writer = csv.writer(f,delimiter="|")
    writer.writerows(result)

In [29]:
curser.close()
conn.close()

In [30]:
# load data to s3
aws_parser = configparser.ConfigParser()
aws_parser.read("pipeline.conf")

['pipeline.conf']

In [31]:
access_key = aws_parser.get("aws_boto_credentials","access_key")
secret_key = aws_parser.get("aws_boto_credentials","secret_key")
bucket_name = aws_parser.get("aws_boto_credentials","bucket_name")

In [32]:
s3 = boto3.client('s3' , 
                 aws_access_key_id = access_key,
                aws_secret_access_key = secret_key)

In [33]:
file = "orders.csv"

In [34]:
s3.upload_file(
file,
bucket_name,
file)